In [1]:
import sys
sys.path.append('..//')
from ioMicro import *

In [2]:
folders = glob.glob(r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\P*')+glob.glob(r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\H0*')
folders

['\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\P1_NeuroD1_FoxJ1_Gfap__set1',
 '\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\P2_GAD1_NRGN_AQP4__set1',
 '\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\P3_PTBP1_PTBP2_REST__set1',
 '\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\H0__set1']

In [3]:
def gene_to_file_color(gene,ifov,folders=folders,ncols=3):
    fovs = np.sort([os.path.basename(fl) for fl in glob.glob(folders[0]+os.sep+'*.zarr')])
    global dic_gn
    dic_gn = {}
    for fld in folders:
        tag = os.path.basename(fld)
        gns = tag.replace('__','_').split('_')[1:-1]
        #print(gns)
        if len(gns)==0:
            for icol in range(ncols):
                dic_gn['bk'+str(icol)]=(fld,icol)
        else:
            for icol,gn in enumerate(gns):
                dic_gn[gn]=(fld,icol)
    icol = dic_gn[gene][1]
    return dic_gn[gene][0]+os.sep+fovs[ifov],dic_gn['bk'+str(icol)][0]+os.sep+fovs[ifov],icol

def get_im_med(icol=0,save_fld_mos = r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\Mosaic_Analysis'):
    im_med = np.load(save_fld_mos+os.sep+'med_col_raw'+str(icol)+'.npz')['im']
    im_med = cv2.blur(im_med,(20,20))
    im_med = im_med/np.median(im_med)
    return im_med

In [4]:
gene_to_file_color('NRGN',30)

('\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\P2_GAD1_NRGN_AQP4__set1\\Conv_zscan__030.zarr',
 '\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\H0__set1\\Conv_zscan__030.zarr',
 1)

In [5]:
list(dic_gn.keys())[:-3]

['NeuroD1', 'FoxJ1', 'Gfap', 'GAD1', 'NRGN', 'AQP4', 'PTBP1', 'PTBP2', 'REST']

In [79]:
fl_sig,fl_bk,icol = gene_to_file_color('PTBP1',305)
im_sig,x,y = read_im(fl_sig,return_pos=True)
im_bk,x,y = read_im(fl_bk,return_pos=True)
im_sig_dapi = np.array(im_sig[-1],dtype=np.float32)
im_bk_dapi = np.array(im_bk[-1],dtype=np.float32)
im_sig = np.array(im_sig[icol],dtype=np.float32)
im_bk = np.array(im_bk[icol],dtype=np.float32)
tzxy,tzxys = get_txyz(im_sig_dapi,im_bk_dapi,sz_norm=30,sz=300)

# slice_bk = tuple([slice(np.max([t,0]),np.min([t,-1])) for t in tzxy])
# slice_sig = tuple([slice(np.max([-t-1,0]),np.min([-t-1,-1])) for t in tzxy])
# im_sig_ = im_sig[slice_sig]-im_bk[slice_bk]
# im_sig_n = norm_slice(im_sig_,s=30)
# im_dapi = im_bk_dapi[slice_bk]


slice_bk = tuple([slice(np.max([t,0]),np.min([t,-1])) for t in tzxy])
slice_sig = tuple([slice(np.max([-t-1,0]),np.min([-t-1,-1])) for t in tzxy])
im_sig_ = (im_sig[slice_sig]-im_bk[slice_bk])/get_im_med(icol)[slice_sig[1:]]
im_sig_n = norm_slice(im_sig_,s=30)
im_dapi = im_bk_dapi[slice_bk]/get_im_med(3)[slice_bk[1:]]

In [106]:
import napari
v = napari.Viewer()
v.add_image(np.max(im_sig_n,axis=0),colormap='yellow',blending='additive')
v.add_image(np.max(im_dapi,axis=0),colormap='bop blue',blending='additive')

<Image layer 'Image [1]' at 0x187c91c6c10>

In [6]:
def get_imsm(gene,ifov,
             redo=False,
             save_fld_mos = r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\Mosaic_Analysis'):
    
    if save_fld_mos is not None:
        if not redo:
            mosaic_fl = save_fld_mos+os.sep+os.sep+str(ifov).zfill(4)+'__'+gene+'.npz'

            if os.path.exists(mosaic_fl):
                im_nc = np.load(mosaic_fl)['im']
                Xh = np.load(mosaic_fl)['Xh']
                im_dapi = np.load(mosaic_fl)['im_dapi_']
                x,y,icol = np.load(mosaic_fl)['xycol']
                return im_nc,Xh,im_dapi,icol,x,y
        
    fl_sig,fl_bk,icol = gene_to_file_color(gene,ifov)
                 
    im_sig,x,y = read_im(fl_sig,return_pos=True)
    im_bk,x,y = read_im(fl_bk,return_pos=True)
    im_sig_dapi = np.array(im_sig[-1],dtype=np.float32)
    im_bk_dapi = np.array(im_bk[-1],dtype=np.float32)
    im_sig = np.array(im_sig[icol],dtype=np.float32)
    im_bk = np.array(im_bk[icol],dtype=np.float32)
    tzxy,tzxys = get_txyz(im_sig_dapi,im_bk_dapi,sz_norm=30,sz=300)
    
    # slice_bk = tuple([slice(np.max([t,0]),np.min([t,-1])) for t in tzxy])
    # slice_sig = tuple([slice(np.max([-t-1,0]),np.min([-t-1,-1])) for t in tzxy])
    # im_sig_ = im_sig[slice_sig]-im_bk[slice_bk]
    # im_sig_n = norm_slice(im_sig_,s=30)
    # im_dapi = im_bk_dapi[slice_bk]
    
    
    slice_bk = tuple([slice(np.max([t,0]),np.min([t,-1])) for t in tzxy])
    slice_sig = tuple([slice(np.max([-t-1,0]),np.min([-t-1,-1])) for t in tzxy])
    im_sig_ = (im_sig[slice_sig]-im_bk[slice_bk])/get_im_med(icol)[slice_sig[1:]]
    im_sig_n = norm_slice(im_sig_,s=30)
    im_dapi = im_bk_dapi[slice_bk]/get_im_med(3)[slice_bk[1:]]
    im_nc = np.max(im_sig_n,axis=0)
    Xh = get_local_maxfast_tensor(im_sig_n,500,im_raw=im_sig_)
    im_dapi_ = np.max(im_dapi,axis=0)
    if save_fld_mos is not None:
        mosaic_fl = save_fld_mos+os.sep+os.sep+str(ifov).zfill(4)+'__'+gene+'.npz'
        np.savez_compressed(mosaic_fl,im=im_nc,Xh=Xh,im_dapi_ = im_dapi_,xycol = [x,y,icol],tzxy=tzxy,tzxys=tzxys)
    
    return im_nc,Xh,im_dapi_,icol,x,y

In [7]:
genes = ['NeuroD1', 'FoxJ1', 'Gfap', 'GAD1', 'NRGN', 'AQP4', 'PTBP1', 'PTBP2', 'REST']

In [15]:
ifovs = [264,265,266,275,276,277,279,280,281,
 263,268,267,274,273,278,283,282,311,
 257,269,270,271,272,285,284,309,310,
 256,251,250,249,287,286,307,308,220]

In [16]:
for ifov in tqdm(ifovs):
    for gene in tqdm(genes):
        im_sig,Xh,im_dapi,icol,x,y = get_imsm(gene,ifov)

100%|███████████████████████████████████████████████████████████████████████████████| 36/36 [3:12:44<00:00, 321.23s/it]


In [13]:
import napari
v = napari.Viewer()
v.add_image(im_sig,colormap='yellow',blending='additive',contrast_limits=[0,5000])
v.add_image(im_dapi,colormap='bop blue',blending='additive')

<Image layer 'im_dapi' at 0x20c8e1f6730>

NameError: name 'im_dapi' is not defined

array([[165., 159., 160., ..., 171., 167., 156.],
       [158., 160., 169., ..., 161., 164., 156.],
       [159., 161., 162., ..., 163., 176., 169.],
       ...,
       [162., 170., 175., ..., 181., 197., 187.],
       [175., 160., 163., ..., 177., 186., 189.],
       [181., 176., 166., ..., 198., 185., 185.]], dtype=float32)

In [5]:
def get_im(gene,ifov= 1277,icol_=None):
    if 'bk' in gene:
        hfld = r'\\BBFISH2\Raw_data_1\KKBB_4_29_2022_full\H0'
        icol = int(gene[2:])
    else:
        hfld,icol = gene_to_file_color(gene)
    if icol_ is not None:
        icol = icol_
    fov = 'Conv_zscan__'+str(ifov).zfill(4)+'.zarr'
    fl = hfld+os.sep+fov
    im,x,y = read_im(fl,return_pos=True)
    return im[icol,1:],icol,x,y

def norm_im_med(im,icol):
    imA,imB = np.load(r'\\BBFISH2\Raw_data_1\KKBB_4_29_2022_full_Analysis_v1\median_col'+str(icol)+'.npy')
    imA= cv2.blur(imA,(50,50))
    imB= cv2.blur(imB,(50,50))
    im_f = (np.array(im,dtype=np.float32)-imA)/imB
    return im_f
def get_imsm(gene,ifov,vmax=3,snorm=30,
             redo=False,
             save_fld_mos = r'\\BBFISH2\Raw_data_1\KKBB_4_29_2022_full_Analysis_review_mosaics'):
    if save_fld_mos is not None:
        if not redo:
            mosaic_fl = save_fld_mos+os.sep+os.sep+str(ifov).zfill(4)+'__'+gene+'.npz'

            if os.path.exists(mosaic_fl):
                im_nc = np.load(mosaic_fl)['im']
                x,y,icol = np.load(mosaic_fl)['xycol']
                return im_nc,icol,x,y
        
    im,icol,x,y = get_im(gene,ifov= ifov,icol_=None)
    im_ = np.array(im,dtype=np.float32)
    im_n = norm_im_med(im_,icol)
    im_n = np.max(norm_slice(im_n,s=snorm),axis=0)
    im_nc = (np.clip(im_n/vmax,0,1)*255).astype(np.uint8)
    
    if save_fld_mos is not None:
        mosaic_fl = save_fld_mos+os.sep+os.sep+str(ifov).zfill(4)+'__'+gene+'.npz'
        np.savez_compressed(mosaic_fl,im=im_nc,xycol = [x,y,icol])
    
    return im_nc,icol,x,y

['\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\P1_NeuroD1_FoxJ1_Gfap__set1',
 '\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\P2_GAD1_NRGN_AQP4__set1',
 '\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\P3_PTBP1_PTBP2_REST__set1']